In [1]:
!pip install rdflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.7 MB/s eta 0:00:00


In [3]:
import csv
from rdflib import Graph, Literal, Namespace, RDF, URIRef, OWL
from rdflib.namespace import XSD

ont = Namespace("http://www.semanticweb.org/addalin/ontologies/2024/3/untitled-ontology-19/")
xsd = Namespace("http://www.w3.org/2001/XMLSchema#")

# Load ontology
g = Graph()
g.parse("ontology_2.ttl", format="turtle")

BELONGS_TO = ont.BELONGS_TO
g.add((BELONGS_TO, RDF.type, OWL.ObjectProperty))

# Read data from stadiums CSV file
with open("country_code.csv", "r", newline="") as csvfile:
    reader = csv.DictReader(csvfile)


    for row in reader:
        alpha2_uri = URIRef(ont[str(row["ISO 3166-1 alpha2"])])
        g.add((alpha2_uri, RDF.type, ont.ISO_3166_1_alpha2))


        alpha3_uri = URIRef(ont[str(row["ISO 3166-1 alpha3"])])
        g.add((alpha3_uri, RDF.type, ont.ISO_3166_1_alpha3))

        numeric_uri = URIRef(ont[str(row["ISO 3166-1 numeric"])])
        g.add((numeric_uri, RDF.type, ont.ISO_3166_1_numeric))

        IOC_uri = URIRef(ont[str(row["IOC"])])
        g.add((IOC_uri, RDF.type, ont.IOC))

        fips_uri = URIRef(ont[str(row["Fips 10"])])
        g.add((fips_uri, RDF.type, ont.Fips_10))

        lp_uri = URIRef(ont[str(row["License Plate"])])
        g.add((lp_uri, RDF.type, ont.License_Plate))

        domain_uri = URIRef(ont[str(row["Domain"])])
        g.add((domain_uri, RDF.type, ont.Domain))
        Country_name = Literal(row["Country"], datatype=XSD.string)
        g.add((domain_uri, BELONGS_TO, Country_name))



        g.add((alpha2_uri, ont.hasDomain, domain_uri))
        g.add((alpha3_uri, ont.hasDomain, domain_uri))
        g.add((numeric_uri, ont.hasDomain, domain_uri))
        g.add((IOC_uri, ont.hasDomain, domain_uri))
        g.add((fips_uri, ont.hasDomain, domain_uri))
        g.add((lp_uri, ont.hasDomain, domain_uri))

        hasCountry_Name = ont.hasCountry_Name
        g.add((hasCountry_Name, RDF.type, OWL.DatatypeProperty))



        Country_name = Literal(row["Country"], datatype=XSD.string)
        g.add((alpha2_uri, hasCountry_Name, Country_name))
        g.add((alpha3_uri, hasCountry_Name, Country_name))
        g.add((numeric_uri, hasCountry_Name, Country_name))
        g.add((IOC_uri, hasCountry_Name, Country_name))
        g.add((fips_uri, hasCountry_Name, Country_name))
        g.add((lp_uri, hasCountry_Name, Country_name))

g.serialize(destination="ontology_2_populated.ttl", format="turtle")

<Graph identifier=Nf6294dfadbe248ecb56afdb6aa4bffe4 (<class 'rdflib.graph.Graph'>)>

In [4]:
import pandas as pd
from rdflib import Graph, Namespace, RDF, OWL, URIRef, Literal, BNode, RDFS

# Load the existing ontology
g = Graph()
ontology_path = 'ontology_2_populated.ttl'
g.parse(ontology_path, format="turtle")

# Define the namespace used in the ontology
ont = Namespace("http://www.semanticweb.org/addalin/ontologies/2024/3/untitled-ontology-19/")
g.bind("ont", ont)

# Define new URIs for additional properties
iso_alpha2_uri = URIRef(ont["hasAlpha_2"])
iso_alpha3_uri = URIRef(ont["hasAlpha_3"])
domain_uri = URIRef(ont["hasDomain"])


# Load the CSV file containing the new dataset
csv_file = "country_code.csv"  # Update this with the actual path to your CSV file
data = pd.read_csv(csv_file)

# Process each row in the dataset
for index, row in data.iterrows():
    country_id = row['Country'].replace(" ", "_")
    country_uri = URIRef(ont[country_id])

    # Ensure the country is modeled in the ontology
    if (country_uri, RDF.type, OWL.Class) not in g:
        g.add((country_uri, RDF.type, OWL.Class))

    # Process ISO Alpha-2 code using BNode for restrictions
    if pd.notna(row['ISO 3166-1 alpha2']):
        alpha_2_name = row['ISO 3166-1 alpha2']
        alpha_2_uri = URIRef(ont[str(alpha_2_name)])
        alpha2_bnode = BNode()
        g.add((alpha2_bnode, RDF.type, OWL.Restriction))
        g.add((alpha2_bnode, OWL.onProperty, iso_alpha2_uri))
        g.add((alpha2_bnode, OWL.hasValue, alpha_2_uri))
        g.add((country_uri, RDFS.subClassOf, alpha2_bnode))

    # Process ISO Alpha-3 code using BNode for restrictions
    if pd.notna(row['ISO 3166-1 alpha3']):
        alpha_3_name = row['ISO 3166-1 alpha3']
        alpha_3_uri = URIRef(ont[str(alpha_3_name)])
        alpha3_bnode = BNode()
        g.add((alpha3_bnode, RDF.type, OWL.Restriction))
        g.add((alpha3_bnode, OWL.onProperty, iso_alpha3_uri))
        g.add((alpha3_bnode, OWL.hasValue, alpha_3_uri))
        g.add((country_uri, RDFS.subClassOf, alpha3_bnode))

    # Process Domain using BNode for restrictions
    if pd.notna(row['Domain']):
        domain_name = row['Domain']
        domain_name_uri = URIRef(ont[str(domain_name)])
        domain_bnode = BNode()
        g.add((domain_bnode, RDF.type, OWL.Restriction))
        g.add((domain_bnode, OWL.onProperty, domain_uri))
        g.add((domain_bnode, OWL.hasValue, domain_name_uri))
        g.add((country_uri, RDFS.subClassOf, domain_bnode))

# Serialize the updated graph to a TTL file
output_ttl_path = 'ontology_2_populated.ttl'
g.serialize(destination=output_ttl_path, format='turtle')

<Graph identifier=N88aca48244b44c1686852dc10ec15e08 (<class 'rdflib.graph.Graph'>)>